# XAI CODE DEMO

## Explainable AI Specialization on Coursera

If you experience high latency while running this notebook, you can open it in Google Colab:

[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/explainable-machine-learning/interpretable-ml/blob/main/rulefit_interpretability.ipynb)

# RuleFit

* RuleFit learns a sparse linear model with the original features AND also a number of new features that are decision rules
* New features that are decision rules capture interactions between the original features
* These features are generated from decision trees  trained to predict the outcome of interest

Steps:
1. Generate Rules
2. Create Sparse Linear Model

In this code demo, we will implement two versions of RuleFit. The first is an implementation via the imodels python library and the second is only using the scikit-learn package. Both implementations use a tree ensemble with gradient boosting and a Lasso linear model.

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder

from imodels import RuleFitRegressor

#### Dataset

We will be using the Diabetes datastet for this demonstration: [LINK](https://www.geeksforgeeks.org/sklearn-diabetes-dataset/)

This dataset is from [Efron, et.al.](https://hastie.su.domains/Papers/LARS/LeastAngle_2002.pdf)

The features in this dataset are age, sex, bmi, blood pressure (bp), and six serum measurements (s1-s6).

In [2]:
# Load the diabetes dataset
diabetes = load_diabetes()
X = pd.DataFrame(diabetes.data, columns=diabetes.feature_names)
y = diabetes.target

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## RuleFit Implementation with *imodels*
[imodels RuleFit algorithm](https://csinva.io/imodels/rule_set/rule_fit.html#imodels.rule_set.rule_fit.RuleFit)

From documentation:
Linear model of tree-based decision rules based on the rulefit algorithm from Friedman and Popescu.

The algorithm can be used for predicting an output vector y given an input matrix X. In the first step a tree ensemble is generated with gradient boosting. The trees are then used to form rules, where the paths to each node in each tree form one rule. A rule is a binary decision if an observation is in a given node, which is dependent on the input features that were used in the splits. The ensemble of rules together with the original input features are then being input in a L1-regularized linear model, also called Lasso, which estimates the effects of each rule on the output target but at the same time estimating many of those effects to zero.

### Initialize and train RuleFitRegressor

The below cell make take a few moments to run, since it is training our model.

In [3]:
# Initialize the RuleFitRegressor
model = RuleFitRegressor()

# Train the model
model.fit(X_train, y_train)

RuleFitRegressor()

### Evaluate Model

In [4]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')


Mean Squared Error: 2997.91


### Extract Rules

In [5]:
rule_df = model.visualize()
rule_df

,rule,coef
3,bp,70.76
27,s5 > -0.03388,1.28
21,bmi <= 0.07625 and s5 <= 0.022,-8.80
25,s3 > -0.01947,-4.40
20,bmi <= 0.0687 and s5 <= 0.01594,-2.13
22,bmi <= 0.01481 and s4 <= 0.06438,-0.34
24,s3 > -0.01579,-1.71
15,s5 <= -0.00017,-3.35
18,bmi <= 0.00619 and s3 > -0.07469 and s5 <= 0.0199,-13.41
14,bmi <= 0.00942 and s4 <= 0.03062,-13.79


In [13]:
rule_df.sort_values(['coef'], ascending=False)

,rule,coef
3,bp,70.76
28,bmi > 0.01804 and bp > 0.01671 and s5 > -0.032,25.01
37,bmi > 0.07355,23.11
31,age > 0.0072 and bmi <= 0.06601 and bp > -0.02461 and s2 <= -0.00351 and s5 > -0.01706,14.05
32,age > -0.05092 and bmi > 0.0105 and s5 > -0.00331,11.10
33,s5 > -0.00017 and s6 > 0.04241,7.39
36,bp > 0.06147 and s5 > 0.01594,7.16
30,s5 > -0.00017 and s6 > 0.03827,6.42
35,bmi > 0.05577 and s5 > -0.00017,5.41
29,bp > 0.02359 and s5 > -0.03019,3.70




---



## Implementing RuleFit without *imodels*

### Train a tree ensemble with gradient boosting

In [6]:
# Train the gradient boosting model
gb = GradientBoostingRegressor(n_estimators=100, max_depth=3, random_state=42)
gb.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

### Extract rules from the tree ensemble

Deduplicate rules

In [7]:
def extract_rules(tree, feature_names):
    rules = []
    tree_ = tree.tree_
    feature = tree_.feature
    threshold = tree_.threshold

    def traverse(node, rule):
        if tree_.feature[node] != -2:
            name = feature_names[feature[node]]
            threshold_value = threshold[node]
            left_rule = rule + [f"{name} <= {threshold_value}"]
            right_rule = rule + [f"{name} > {threshold_value}"]
            traverse(tree_.children_left[node], left_rule)
            traverse(tree_.children_right[node], right_rule)
        else:
            rules.append(rule)

    traverse(0, [])
    return rules

# Extract rules from all trees
rules = []
for estimator in gb.estimators_:
    for tree in estimator:
        rules.extend(extract_rules(tree, X.columns))

# Deduplicate rules
rules = list(map(list, {tuple(rule) for rule in rules}))


### Convert rules to feature matrix

In [14]:
def rule_to_feature_matrix(rules, X):
    feature_matrix = np.zeros((X.shape[0], len(rules)), dtype=int)
    for i, rule in enumerate(rules):
        rule_conditions = " & ".join(rule)
        feature_matrix[:, i] = X.eval(rule_conditions).astype(int)
    return feature_matrix

rule_features_train = rule_to_feature_matrix(rules, X_train)
rule_features_test = rule_to_feature_matrix(rules, X_test)
rule_features_train

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

### Train Lasso model

Combine rule-based features with original features

In [9]:
# Combine original features and rule-based features
X_train_combined = np.hstack([X_train, rule_features_train])
X_test_combined = np.hstack([X_test, rule_features_test])

# Train the Lasso model
lasso = Lasso(alpha=0.1)
lasso.fit(X_train_combined, y_train)

Lasso(alpha=0.1)

### Evaluate model

In [10]:
# Make predictions on the test set
y_pred = lasso.predict(X_test_combined)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.2f}')

Mean Squared Error: 3702.89


### Extract Rules

In [11]:
# Get the coefficients and feature names (original features + rule-based features)
coefs = lasso.coef_
original_feature_names = X.columns.tolist()
rule_feature_names = [f"Rule {i+1}" for i in range(len(rules))]
all_feature_names = original_feature_names + rule_feature_names

# Sort coefficients and feature names by absolute coefficient value
coef_feature_pairs = sorted(zip(coefs, all_feature_names), key=lambda x: abs(x[0]), reverse=True)

# Print rules
for coef, feature_name in coef_feature_pairs:
    if coef != 0:
        if "Rule" in feature_name:
            rule_index = int(feature_name.split()[1]) - 1
            rule_description = " AND ".join(rules[rule_index])
            print(f"{feature_name}: Coefficient = {coef:.4f}, Rule = {rule_description}")
        else:
            print(f"{feature_name}: Coefficient = {coef:.4f}")


Rule 430: Coefficient = 80.5573, Rule = s6 > 0.04241442494094372 AND s5 > 0.01368608744814992 AND s5 <= 0.015194719657301903
Rule 122: Coefficient = 55.4621, Rule = bp <= -0.03837750665843487 AND s4 <= 0.10755758732557297 AND bmi > 0.11720352992415428
Rule 522: Coefficient = 44.0636, Rule = bp <= 0.09818317741155624 AND s1 <= -0.024272182025015354 AND s1 > -0.028400040231645107
Rule 505: Coefficient = 43.7404, Rule = s5 <= -0.0010538420465309173 AND s2 > 0.017944753170013428 AND age <= -0.0581863634288311
Rule 93: Coefficient = -40.2734, Rule = s6 <= -0.003148751042317599 AND bp > 0.016708109062165022 AND s5 <= -0.03781251050531864
Rule 497: Coefficient = -32.8536, Rule = s4 <= 0.05441997013986111 AND s5 > 0.08501111716032028 AND bp <= 0.03220093855634332
Rule 18: Coefficient = -30.4212, Rule = age <= 0.007199329789727926 AND s2 > 0.016535584814846516 AND age > 0.003566791128832847
Rule 177: Coefficient = 29.8528, Rule = s5 <= 0.016671447083353996 AND bp <= 0.054579468443989754 AND bp 